In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import pandas as pd
import numpy as np
import os

def input_data(path_data):
    if path_data.endswith('.csv'):
        data = pd.read_csv(path_data)
    elif path_data.endswith('.xlsx'):
        data = pd.read_excel(path_data)
    return data

def compute_features(df):

    # Tạo lag
    df['lag_1'] = df['Close']
    df['lag_5'] = df['Close'].shift(4)
    df['lag_10'] = df['Close'].shift(9)

    df['r1'] = np.log(df['Close']/df['Close'].shift(1))
    df['r2'] = np.log(df['Close'].shift(1)/df['Close'].shift(2))
    df['r3'] = np.log(df['Close'].shift(2)/df['Close'].shift(3))
    df['r4'] = np.log(df['Close'].shift(3)/df['Close'].shift(4))
    
    df['r5'] = np.log(df['High'] / df['Open'])
    df['r6'] = np.log(df['High'] / df['Open'].shift(1))
    df['r7'] = np.log(df['High'] / df['Open'].shift(2))
    df['r8'] = np.log(df['High'] / df['Open'].shift(3))

    df['r9'] = np.log(df['High'].shift(1) / df['Open'].shift(1))
    df['r10'] = np.log(df['High'].shift(2) / df['Open'].shift(2))
    df['r11'] = np.log(df['High'].shift(3) / df['Open'].shift(3))

    df['r12'] = np.log(df['Low'] / df['Open'])
    df['r13'] = np.log(df['Low'].shift(1) / df['Open'].shift(1))
    df['r14'] = np.log(df['Low'].shift(2) / df['Open'].shift(2))
    df['r15'] = np.log(df['Low'].shift(3) / df['Open'].shift(3))
    df['RSI'] = compute_rsi(df['Close'], period=14)
    df['Momentum'] = df['Close'] - df['Close'].shift(10)
    df['TrueRange'] = compute_true_range(df)
    df['ATR'] = df['TrueRange'].rolling(window=14).mean()
    df['ParabolicSAR'] = compute_parabolic_sar(df, max_acc=0)


    # Xử lý dữ liệu vô hạn
    df.replace([np.inf, -np.inf], 0, inplace=True)
    # Tạo cột dự đoán    
    df['Close'] = df['Close'].shift(-1)
        # Loại bỏ cột không cần thiết
    df.drop(columns=['High', 'Low', 'Open', 'Volume'], errors='ignore', inplace=True)

    # Điền giá trị thiếu
    df.fillna(method='bfill', inplace=True)
    return df

def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_true_range(df):
    tr1 = df['High'] - df['Low']
    tr2 = abs(df['High'] - df['Close'].shift(1))
    tr3 = abs(df['Low'] - df['Close'].shift(1))
    return pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

def compute_parabolic_sar(df, acc=0.02, max_acc=0.2):
    sar = df['Close'].copy()
    return sar  # Placeholder, implement SAR logic if needed

folder_path = "../Data_FTSE_01012010_30122024/"

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    df = input_data(file_path)
    df = compute_features(df)
    print(df)
    # break
    df.to_csv(rf"G:\FintechResearch\portfolio-formation-with-ltsm\Data_FTSE_model\{file}", index=False)


C:\Users\dntth\AppData\Local\Temp\ipykernel_2760\372622173.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


            Date        Close        lag_1        lag_5       lag_10  \
0     2010-01-04    64.822250    63.895664    63.895664    63.895664   
1     2010-01-05    64.474770    64.822250    63.895664    63.895664   
2     2010-01-06    65.517181    64.474770    63.895664    63.895664   
3     2010-01-07    68.837425    65.517181    63.895664    63.895664   
4     2010-01-08    69.107681    68.837425    63.895664    63.895664   
...          ...          ...          ...          ...          ...   
3782  2024-12-20  4941.818359  4993.440430  5096.684570  6226.413086   
3783  2024-12-23  4912.036621  4941.818359  5049.033203  5352.809082   
3784  2024-12-24  4906.080078  4912.036621  5128.452148  5261.478027   
3785  2024-12-27  4895.160156  4906.080078  4953.730957  5209.855957   
3786  2024-12-30          NaN  4895.160156  4993.440430  5158.233887   

            r1        r2        r3        r4        r5  ...       r11  \
0     0.014397  0.014397  0.014397  0.014397  0.067823  ...  0

C:\Users\dntth\AppData\Local\Temp\ipykernel_2760\372622173.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


            Date       Close       lag_1       lag_5      lag_10        r1  \
0     2010-01-04  186.400925  175.426987  175.426987  175.426987  0.060677   
1     2010-01-05  191.966064  186.400925  175.426987  175.426987  0.060677   
2     2010-01-06  197.281143  191.966064  175.426987  175.426987  0.029419   
3     2010-01-07  200.438965  197.281143  175.426987  175.426987  0.027311   
4     2010-01-08  198.219055  200.438965  175.426987  175.426987  0.015880   
...          ...         ...         ...         ...         ...       ...   
3782  2024-12-20  261.600006  260.250000  270.299988  267.549988  0.000000   
3783  2024-12-23  263.549988  261.600006  264.549988  264.200012  0.005174   
3784  2024-12-24  264.899994  263.549988  267.049988  266.649994  0.007426   
3785  2024-12-27  264.750000  264.899994  260.250000  270.700012  0.005109   
3786  2024-12-30         NaN  264.750000  260.250000  269.399994 -0.000566   

            r2        r3        r4        r5  ...       r11    

C:\Users\dntth\AppData\Local\Temp\ipykernel_2760\372622173.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


...

C:\Users\dntth\AppData\Local\Temp\ipykernel_2760\372622173.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


            Date       Close       lag_1       lag_5      lag_10        r1  \
0     2010-01-04  282.161224  287.496490  287.496490  287.496490 -0.018732   
1     2010-01-05  276.524078  282.161224  287.496490  287.496490 -0.018732   
2     2010-01-06  276.490570  276.524078  287.496490  287.496490 -0.020181   
3     2010-01-07  279.141327  276.490570  287.496490  287.496490 -0.000121   
4     2010-01-08  280.416534  279.141327  287.496490  287.496490  0.009541   
...          ...         ...         ...         ...         ...       ...   
3782  2024-12-20  366.000000  366.399994  373.100006  364.799988 -0.005715   
3783  2024-12-23  368.500000  366.000000  374.000000  368.600006 -0.001092   
3784  2024-12-24  366.899994  368.500000  372.899994  370.200012  0.006807   
3785  2024-12-27  365.600006  366.899994  368.500000  370.600006 -0.004351   
3786  2024-12-30         NaN  365.600006  366.399994  372.000000 -0.003549   

            r2        r3        r4        r5  ...       r11    

C:\Users\dntth\AppData\Local\Temp\ipykernel_2760\372622173.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


            Date       Close       lag_1       lag_5      lag_10        r1  \
0     2010-06-29    1.588667    1.592667    1.592667    1.592667 -0.002515   
1     2010-06-30    1.464000    1.588667    1.592667    1.592667 -0.002515   
2     2010-07-01    1.280000    1.464000    1.592667    1.592667 -0.081723   
3     2010-07-02    1.074000    1.280000    1.592667    1.592667 -0.134312   
4     2010-07-06    1.053333    1.074000    1.592667    1.592667 -0.175470   
...          ...         ...         ...         ...         ...       ...   
3646  2024-12-23  462.279999  430.600006  479.859985  400.989990  0.022404   
3647  2024-12-24  454.130005  462.279999  440.130005  424.769989  0.070991   
3648  2024-12-26  431.660004  454.130005  436.170013  418.100006 -0.017787   
3649  2024-12-27  417.410004  431.660004  421.059998  436.230011 -0.050745   
3650  2024-12-30         NaN  417.410004  430.600006  463.019989 -0.033569   

            r2        r3        r4        r5  ...       r11    

C:\Users\dntth\AppData\Local\Temp\ipykernel_2760\372622173.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

def select_features(X, y, n_features_to_select=20, feature_range=(-1, 1)):
    """
    Hàm này thực hiện chọn đặc trưng sử dụng RFE với Logistic Regression và chuẩn hóa dữ liệu.

    Parameters:
    - X (pd.DataFrame): DataFrame chứa các đặc trưng đầu vào.
    - y (pd.Series): Series chứa biến mục tiêu (lợi nhuận tại thời điểm t+1).
    - n_features_to_select (int): Số lượng đặc trưng cần chọn (mặc định là 20).
    - feature_range (tuple): Phạm vi giá trị để chuẩn hóa dữ liệu (mặc định là (-1, 1)).

    Returns:
    - selected_features (list): Danh sách các đặc trưng được chọn.
    - X_selected (pd.DataFrame): DataFrame chứa các đặc trưng được chọn.
    - X_scaled (pd.DataFrame): DataFrame chứa các đặc trưng được chọn và đã chuẩn hóa.
    """

    # 1. Chuẩn hóa dữ liệu
    scaler = MinMaxScaler(feature_range=feature_range)
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    # 2. Chọn đặc trưng sử dụng RFE với Logistic Regression
    model = LogisticRegression()
    rfe = RFE(model, n_features_to_select=n_features_to_select)
    rfe.fit(X_scaled, y)

    # 3. Lấy các đặc trưng được chọn
    selected_features = X.columns[rfe.support_].tolist()
    X_selected = X[selected_features]
    X_scaled_selected = X_scaled[selected_features]

    return selected_features, X_selected, X_scaled_selected

# Ví dụ sử dụng hàm
if __name__ == "__main__":
    # Giả sử X là DataFrame chứa các đặc trưng và y là Series chứa biến mục tiêu
    # Ví dụ:
    # X = pd.DataFrame({
    #     'Open': [...],
    #     'Close': [...],
    #     'High': [...],
    #     'Low': [...],
    #     'Volume': [...],
    #     'RSI': [...],
    #     'MoM': [...],
    #     'TR': [...],
    #     'ATR': [...],
    #     'Parabolic_SAR': [...],
    #     # Thêm các đặc trưng khác...
    # })
    # y = pd.Series([...])  # Lợi nhuận tại thời điểm t+1

    # Gọi hàm để chọn đặc trưng
    selected_features, X_selected, X_scaled_selected = select_features(X, y)

    # In kết quả
    print("Selected Features:", selected_features)
    print("\nSelected Features DataFrame:")
    print(X_selected.head())
    print("\nScaled Selected Features DataFrame:")
    print(X_scaled_selected.head())

In [1]:
import pandas as pd
import numpy as np

# Tạo dữ liệu mẫu cho 10 ngày
data = {
    'Date': pd.date_range(start='2023-10-01', periods=10, freq='D'),  # Ngày từ 01/10/2023 đến 10/10/2023
    'Open': np.random.uniform(100, 110, 10).round(2),  # Giá mở cửa ngẫu nhiên từ 100 đến 110
    'High': np.random.uniform(110, 120, 10).round(2),  # Giá cao nhất ngẫu nhiên từ 110 đến 120
    'Low': np.random.uniform(90, 100, 10).round(2),    # Giá thấp nhất ngẫu nhiên từ 90 đến 100
    'Close': np.random.uniform(100, 110, 10).round(2), # Giá đóng cửa ngẫu nhiên từ 100 đến 110
    'Volume': np.random.randint(1000, 5000, 10)        # Khối lượng giao dịch ngẫu nhiên từ 1000 đến 5000
}

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data)

# In ra dữ liệu
print("Sample Data:")
print(df)

Sample Data:
        Date    Open    High    Low   Close  Volume
0 2023-10-01  109.23  111.93  97.95  103.76    2889
1 2023-10-02  101.08  110.47  93.99  101.79    1634
2 2023-10-03  103.55  114.45  95.73  102.53    4330
3 2023-10-04  106.09  111.48  95.19  102.39    1480
4 2023-10-05  102.51  119.07  93.99  104.14    1633
5 2023-10-06  103.78  111.78  90.77  101.99    4903
6 2023-10-07  101.47  113.92  99.24  101.54    1720
7 2023-10-08  100.34  115.32  94.16  105.92    3497
8 2023-10-09  107.43  112.57  96.84  108.44    1982
9 2023-10-10  100.49  117.79  98.18  104.11    3155


In [5]:
# Thiết lập các tham số
train_size = 5  # Kích thước tập huấn luyện (5 ngày)
test_size = 2   # Kích thước tập kiểm tra (2 ngày)
step_size = 2   # Bước trượt (2 ngày)

# Tạo các study periods
study_periods = []
for i in range(0, len(df) - train_size - test_size + 1, step_size):
    train_start = i
    train_end = i + train_size
    test_start = train_end
    test_end = test_start + test_size
    
    train_set = df.iloc[train_start:train_end]
    test_set = df.iloc[test_start:test_end]
    
    study_periods.append((train_set, test_set))
# In ra số lượng study periods
print(f"Total study periods: {len(study_periods)}")
for i in range(len(study_periods)):
    # Ví dụ: In ra thông tin của study period đầu tiên
    train_set, test_set = study_periods[i]
    print("\nTraining set (first study period):")
    print(train_set[['Date', 'Close']])
    print("\nTesting set (first study period):")
    print(test_set[['Date', 'Close']])

Total study periods: 2

Training set (first study period):
        Date   Close
0 2023-10-01  103.76
1 2023-10-02  101.79
2 2023-10-03  102.53
3 2023-10-04  102.39
4 2023-10-05  104.14

Testing set (first study period):
        Date   Close
5 2023-10-06  101.99
6 2023-10-07  101.54

Training set (first study period):
        Date   Close
2 2023-10-03  102.53
3 2023-10-04  102.39
4 2023-10-05  104.14
5 2023-10-06  101.99
6 2023-10-07  101.54

Testing set (first study period):
        Date   Close
7 2023-10-08  105.92
8 2023-10-09  108.44
